In [ ]:
import time

In [ ]:
# Domain Space
from hyperopt import hp
from hyperopt.pyll.base import scope

In [ ]:
%run targets.ipynb

In [ ]:
from abc import ABCMeta, abstractmethod

class Model:
    __metaclass__ = ABCMeta
    
    def init_model(self, params):
        self.model = self.get_model_class()
        params.update(self.get_default_params())
        params = self.resolve_params(params)
        self.model = self.model(**params)
        return self.model
    
    @abstractmethod
    def get_model_class(self):
        pass
    
    def get_default_params(self):
        return {}
    
    def eval(self, eval_set, eval_metrics=None, verbose=False):
        if eval_metrics == None:
            eval_metrics = self.get_eval_metrics()
        
        result = pd.DataFrame({metric_name:[] for metric_name in eval_metrics})
        for eval_name, eval_data in eval_set.iteritems():
            y_pred = self.predict(eval_data[1])
            y_true = eval_data[2]
            
            metric_scores = eval_class_metrics(y_true, y_pred, metrics=eval_metrics)
            result = result.append(pd.Series(metric_scores, name=eval_name))
            
        return result
    
    def get_eval_metrics(self):
        return ['auc', 'f1', 'acc', 'log_loss']
    
    def get_target_metric(self):
        return 'acc'
    
    @abstractmethod
    def fit(self, X, y, eval_set, eval_names, eval_metrics, verbose=False):
        pass
    
    @abstractmethod
    def predict(self):
        pass
    
    def resolve_params(self, params):
        params_ = dict(params)
        for k, v in params.iteritems():
            if isinstance(v, dict):
                params_.update(self.resolve_params(v))
        return params_

In [ ]:
class NB(Model):
    def __init__(self):
        self.model_type = 'NB'
        
        self.hyperopt_scope = {
            'priors' : hp.choice('priors', [None, [0.5, 0.5]])
        }
        
    def get_model_class(self):
        from sklearn.naive_bayes import GaussianNB
        return GaussianNB
    
    def fit(self, X, y, eval_set, eval_metrics=None, verbose=False):
        self.model = self.model.fit(X, y)
        return self.eval(eval_set, eval_metrics=eval_metrics, verbose=verbose)
        
    def predict(self, X):
        return self.model.predict_proba(X)[:, 1]
    
    @staticmethod
    def get_attributes(model):
        return {}

In [ ]:
class RFC(Model):
    def __init__(self):
        self.model_type = 'RFC'
        
        self.hyperopt_scope = {
            'n_estimators'            : hp.choice('n_estimators', [3, 30, 100]),
            'max_features'            : hp.choice('max_features', ['log2', 'sqrt']),
            'criterion'               : hp.choice('criterion', ['gini', 'entropy']),
            'bootstrap'               : hp.choice('bootstrap', [True, False])
        }
        
        self.grid_scope = {
            'n_estimators' : [10],
            'max_features' : ['log2', 'sqrt'],
            'criterion'    : ['entropy'],
        }
        
    def get_model_class(self):
        from sklearn.ensemble import RandomForestClassifier
        return RandomForestClassifier
    
    def get_default_params(self):
        return {
            'n_jobs'                  : -1,
            'random_state'            : 0,
            'verbose'                 : 1
        }
    
    def fit(self, X, y, eval_set, eval_metrics=None, verbose=False):
        self.model = self.model.fit(X, y)
        print('model fitted')
        return self.eval(eval_set, eval_metrics=eval_metrics, verbose=verbose)
    
    def predict(self, X):
        return self.model.predict_proba(X)[:, 1]
    
    @staticmethod
    def get_attributes(model):
        return {
            'feature_importances_': model.feature_importances_
        }

In [ ]:
class BayesR(Model):
    def __init__(self):
        self.model_type = 'BayesR'
        
        self.hyperopt_scope = {
            'tol'            : hp.choice('tol', [0.001, 0.0001, 0.00001]),
            'alpha_1'        : hp.choice('alpha_1', [0.00001, 0.000001, 0.0000001]),
            'alpha_2'        : hp.choice('alpha_2', [0.00001, 0.000001, 0.0000001]),
            'lambda_1'       : hp.choice('lambda_1', [0.00001, 0.000001, 0.0000001]),
            'lambda_2'       : hp.choice('lambda_2', [0.00001, 0.000001, 0.0000001])
        }
        
    def get_model_class(self):
        from sklearn.linear_model import BayesianRidge
        return BayesianRidge
    
    def get_default_params(self):
        return {
            'n_iter'                  : 300,
            'compute_score'           : False,
            'copy_X'                  : False,
            'fit_intercept'           : True,
            'normalize'               : True,
            'verbose'                 : True
        }
    
    def fit(self, X, y, eval_set, eval_metrics=None, verbose=False):
        self.model = self.model.fit(X, y)
        return self.eval(eval_set, eval_metrics=eval_metrics, verbose=verbose)
    
    def predict(self, X):
        return self.model.predict(X)
    
    @staticmethod
    def get_attributes(model):
        return {}

In [ ]:
class LR(Model):
    def __init__(self):
        self.model_type = 'LR'
        
        self.hyperopt_scope = {
            'penalty'        : hp.choice('penalty', [
                {
                    'penalty': 'l2',
                    'solver': 'sag'
                }
                # ,
                # {
                #     'penalty': 'l1',
                #     'solver': 'liblinear'
                # }
            ]),
            'tol'        : hp.choice('tol', [0.001, 0.0001, 0.00001]),
            'C'          : hp.choice('C', [3, 10, 30, 100])
        }
        
        self.grid_scope = {
            'penalty'      : ['l2'],
            'solver'       : ['sag'],
            'C'            : [3, 10, 30, 100]
        }
        
    def get_model_class(self):
        from sklearn.linear_model import LogisticRegression
        return LogisticRegression
    
    def get_default_params(self):
        return {
            'dual'              : False,
            'fit_intercept'     : True,
            'intercept_scaling' : 1,
            'max_iter'          : 100,
            'multi_class'       : 'ovr',
            'class_weight'      : None,
            'random_state'      : 0,
            'verbose'           : 1,
            'warm_start'        : False,
            'n_jobs'            : -1
        }
    
    def fit(self, X, y, eval_set, eval_metrics=None, verbose=False):
        self.model = self.model.fit(X, y)
        return self.eval(eval_set, eval_metrics=eval_metrics, verbose=verbose)
    
    def predict(self, X):
        return self.model.predict_proba(X)[:, 1]
    
    @staticmethod
    def get_attributes(model):
        return {
            'n_iter_': model.n_iter_
        }

In [ ]:
class SVC(Model):
    def __init__(self):
        self.model_type = 'SVC'
        
        self.hyperopt_scope = {
            'kernel'     : hp.choice('kernel', [
                {
                    'kernel': 'linear'
                },
                {
                    'kernel': 'poly',
                    'degree': hp.choice('degree', [1,2,3,4]),
                    'coef0' : hp.choice('coef0_poly', [0, 1, 3, 10])
                },
                {
                    'kernel': 'rbf'
                },
                {
                    'kernel': 'sigmoid',
                    'coef0' : hp.choice('coef0_sig', [0, 1, 3, 10])
                }
            ]),
            'shrinking'    : hp.choice('shrinking', [True, False]),
            'probability'  : hp.choice('probability', [True, False]),
            'tol'        : hp.choice('tol', [0.00001, 0.000001, 0.0000001]),
            'C'          : hp.choice('C', [0.1, 0.3, 1, 3, 10])
        }
        
    def get_model_class(self):
        from sklearn.svm import SVC
        return SVC
    
    def get_default_params(self):
        return {
            'gamma'             : 'auto',
            'max_iter'          : 1000,
            'random_state'      : 0,
            'verbose'           : 1
        }
    
    def fit(self, X, y, eval_set, eval_metrics=None, verbose=False):
        self.model = self.model.fit(X, y)
        return self.eval(eval_set, eval_metrics=eval_metrics, verbose=verbose)
    
    def predict(self, X):
        return self.model.predict(X)
    
    @staticmethod
    def get_attributes(model):
        return {}